In [6]:
from os import environ, path, walk
from pocketsphinx import *
from sphinxbase import *
import fnmatch
import json
from jiwer import wer
import math

relative_path = "../../td_corpus_digits"
pwd = os.getcwd()
previous = len("4NoiseLevel")
pwd = pwd[:len(pwd)-previous]
#wer -i SNR35dB_Manrulen.ref SNR35dB_Manrulen.hyp > SNR35dB_Manrulen.res

In [7]:
def find_files(filename, search_path):
   result = []

   # Wlaking top-down from the root
   for root, dir, files in os.walk(search_path):
      for file in files:
         if file.__contains__(".raw"):        
            result.append(os.path.join(root, file))
   return result

def calculate_CI(wer, noSen):
    return 1.960 * math.sqrt((wer*(1-wer))/noSen)

def create_decoder_digits(dic, grammar, rule):
    config = Decoder.default_config()
    config.set_string('-hmm', pwd +'ps_data/model/en-us')  # acoustic model
    config.set_string('-dict', pwd +'ps_data/lex/' + dic)  # lexicon / dictionary
    decoder_digit = Decoder(config)

    # Now we use a custom language model
    # Prepare the grammar to be used
    jsgf = Jsgf(pwd +'ps_data/jsgf/' + grammar)  # load the grammar file
    rule = jsgf.get_rule('digits.' + rule)  # choose the rule
    fsg = jsgf.build_fsg(rule, decoder_digit.get_logmath(), 7.5)  # build the grammar rule
    fsg.writefile('../outputs/' + grammar)  # write the compiled grammar rule as an external file

    # Now set the fsg grammar rule in the decoder
    decoder_digit.set_fsg("../outputs/" + grammar, fsg)  # load the pre-recorded compiled grammar rule in the decoder
    decoder_digit.set_search("../outputs/" + grammar)  # and set it as the grammar to use

    return decoder_digit

In [8]:
def rundecoder(file_path, decoder):
    # Start the decoder
    decoder.start_utt()

    # Open the file to decode
    stream = open(file_path, 'rb')
    uttbuf = stream.read(-1)

    # Process the file with the decoder
    if uttbuf:
        decoder.process_raw(uttbuf, False, True)
    else:
        print("Error reading speech data")
        exit()
    decoder.end_utt()

    # test for empty hypothesis and replace the output with an empty string if needed
    if decoder.hyp() is None:
        best_hypothesis = ''
    else:
        best_hypothesis = decoder.hyp().hypstr

    #reading ref file for WER
    with open(file_path[:len(file_path)-3] + "ref") as f:
        actualY = f.readlines()
    
    wordErrorRate = wer(actualY, best_hypothesis)

    return {
    "fileName": file_path.split("/")[-1],
    "prediction": best_hypothesis,
    "actual": actualY[0].replace("\n", ""), 
    # "confidence": decoder.get_logmath().exp(decoder.hyp().prob),
    "wer" : wordErrorRate}

In [9]:
def processWav(waves, rule, group=""):
    decoder = create_decoder_digits("digits.dict", "digits.jsgf", rule)
    ruleN_result = []
    for wav in waves:
        ruleN_result.append(rundecoder(wav, decoder))

    #this is to dump hyp
    with open(group + rule + '.hyp', 'w') as f:
        for i in ruleN_result:
            f.write(i["prediction"])
            f.write("\n")

    #this is to dump ref
    with open(group + rule + '.ref', 'w') as f:
        for i in ruleN_result:
            f.write(i["actual"])
            f.write("\n")

In [11]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq1digit_200_files/"
man1digit35 = find_files(".raw",folder_path)
processWav(man1digit35, "rule1", group="35_")
folder_path = pwd + "td_corpus_digits/SNR25dB/man/seq1digit_200_files/"
man1digit35 = find_files(".raw",folder_path)
processWav(man1digit35, "rule1", group="25_")
folder_path = pwd + "td_corpus_digits/SNR15dB/man/seq1digit_200_files/"
man1digit35 = find_files(".raw",folder_path)
processWav(man1digit35, "rule1", group="15_")
folder_path = pwd + "td_corpus_digits/SNR05dB/man/seq1digit_200_files/"
man1digit35 = find_files(".raw",folder_path)
processWav(man1digit35, "rule1", group="05_")

INFO: pocketsphinx.c(151): Parsed model-specific feature parameters from /home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/model/en-us/feat.params
Current configuration:
[NAME]			[DEFLT]		[VALUE]
-agc			none		none
-agcthresh		2.0		2.000000e+00
-allphone				
-allphone_ci		yes		yes
-alpha			0.97		9.700000e-01
-ascale			20.0		2.000000e+01
-aw			1		1
-backtrace		no		no
-beam			1e-48		1.000000e-48
-bestpath		yes		yes
-bestpathlw		9.5		9.500000e+00
-ceplen			13		13
-cmn			live		batch
-cmninit		40,3,-1		41.00,-5.29,-0.12,5.09,2.48,-4.07,-1.37,-1.78,-5.08,-2.05,-6.45,-1.42,1.17
-compallsen		no		no
-dict					/home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/lex/digits.dict
-dictcase		no		no
-dither			no		no
-doublebw		no		no
-ds			1		1
-fdict					
-feat			1s_c_d_dd	1s_c_d_dd
-featparams				
-fillprob		1e-8		1.000000e-08
-frate			100		100
-fsg					
-fsgusealtpron		yes		yes
-fsgusefiller		yes		yes
-fwdflat		yes		yes
-fwdflatbeam		1e-64		1.000000e-64
-fwdflatefwid		4		4

In [14]:
conf_interval_rule135 = calculate_CI(0.005, 200)
print(conf_interval_rule135*100)
conf_interval_rule125 = calculate_CI(0.01, 200)
print(conf_interval_rule125*100)
conf_interval_rule115 = calculate_CI(0.115, 200)
print(conf_interval_rule115*100)
conf_interval_rule105 = calculate_CI(0.600, 200)
print(conf_interval_rule105*100)

0.9775469298197402
1.3789822333880883
4.421418550646386
6.789639165669999


In [12]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq3digits_100_files/"
man3digit35 = find_files(".raw",folder_path)
processWav(man3digit35, "rule3", group="35_")
folder_path = pwd + "td_corpus_digits/SNR25dB/man/seq3digits_100_files/"
man3digit25 = find_files(".raw",folder_path)
processWav(man3digit25, "rule3", group="25_")
folder_path = pwd + "td_corpus_digits/SNR15dB/man/seq3digits_100_files/"
man3digit15 = find_files(".raw",folder_path)
processWav(man3digit15, "rule3", group="15_")
folder_path = pwd + "td_corpus_digits/SNR05dB/man/seq3digits_100_files/"
man3digit05 = find_files(".raw",folder_path)
processWav(man3digit05, "rule3", group="05_")

INFO: pocketsphinx.c(151): Parsed model-specific feature parameters from /home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/model/en-us/feat.params
Current configuration:
[NAME]			[DEFLT]		[VALUE]
-agc			none		none
-agcthresh		2.0		2.000000e+00
-allphone				
-allphone_ci		yes		yes
-alpha			0.97		9.700000e-01
-ascale			20.0		2.000000e+01
-aw			1		1
-backtrace		no		no
-beam			1e-48		1.000000e-48
-bestpath		yes		yes
-bestpathlw		9.5		9.500000e+00
-ceplen			13		13
-cmn			live		batch
-cmninit		40,3,-1		41.00,-5.29,-0.12,5.09,2.48,-4.07,-1.37,-1.78,-5.08,-2.05,-6.45,-1.42,1.17
-compallsen		no		no
-dict					/home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/lex/digits.dict
-dictcase		no		no
-dither			no		no
-doublebw		no		no
-ds			1		1
-fdict					
-feat			1s_c_d_dd	1s_c_d_dd
-featparams				
-fillprob		1e-8		1.000000e-08
-frate			100		100
-fsg					
-fsgusealtpron		yes		yes
-fsgusefiller		yes		yes
-fwdflat		yes		yes
-fwdflatbeam		1e-64		1.000000e-64
-fwdflatefwid		4		4

In [15]:
conf_interval_rule335 = calculate_CI(0.01667, 100)
print(conf_interval_rule335*100)
conf_interval_rule325 = calculate_CI(0.01667, 100)
print(conf_interval_rule325*100)
conf_interval_rule315 = calculate_CI(0.0900, 100)
print(conf_interval_rule315*100)
conf_interval_rule305 = calculate_CI(0.64333, 100)
print(conf_interval_rule305*100)

2.5094209292536003
2.5094209292536003
5.609162504331641
9.388717340732757


In [13]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq5digits_100_files/"
man5digit35 = find_files(".raw",folder_path)
processWav(man5digit35, "rule5", group="35_")
folder_path = pwd + "td_corpus_digits/SNR25dB/man/seq5digits_100_files/"
man5digit25 = find_files(".raw",folder_path)
processWav(man5digit25, "rule5", group="25_")
folder_path = pwd + "td_corpus_digits/SNR15dB/man/seq5digits_100_files/"
man5digit15 = find_files(".raw",folder_path)
processWav(man5digit15, "rule5", group="15_")
folder_path = pwd + "td_corpus_digits/SNR05dB/man/seq5digits_100_files/"
man5digit05 = find_files(".raw",folder_path)
processWav(man5digit05, "rule5", group="05_")

INFO: pocketsphinx.c(151): Parsed model-specific feature parameters from /home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/model/en-us/feat.params
Current configuration:
[NAME]			[DEFLT]		[VALUE]
-agc			none		none
-agcthresh		2.0		2.000000e+00
-allphone				
-allphone_ci		yes		yes
-alpha			0.97		9.700000e-01
-ascale			20.0		2.000000e+01
-aw			1		1
-backtrace		no		no
-beam			1e-48		1.000000e-48
-bestpath		yes		yes
-bestpathlw		9.5		9.500000e+00
-ceplen			13		13
-cmn			live		batch
-cmninit		40,3,-1		41.00,-5.29,-0.12,5.09,2.48,-4.07,-1.37,-1.78,-5.08,-2.05,-6.45,-1.42,1.17
-compallsen		no		no
-dict					/home/user/Documents/GitHub/ASR/lab_material/Assignment/ps_data/lex/digits.dict
-dictcase		no		no
-dither			no		no
-doublebw		no		no
-ds			1		1
-fdict					
-feat			1s_c_d_dd	1s_c_d_dd
-featparams				
-fillprob		1e-8		1.000000e-08
-frate			100		100
-fsg					
-fsgusealtpron		yes		yes
-fsgusefiller		yes		yes
-fwdflat		yes		yes
-fwdflatbeam		1e-64		1.000000e-64
-fwdflatefwid		4		4

In [16]:
conf_interval_rule535 = calculate_CI(0.0100, 100)
print(conf_interval_rule535*100)
conf_interval_rule525 = calculate_CI(0.0180, 100)
print(conf_interval_rule525*100)
conf_interval_rule515 = calculate_CI(0.0620, 100)
print(conf_interval_rule515*100)
conf_interval_rule505 = calculate_CI(0.5340, 100)
print(conf_interval_rule505*100)

1.950175376728975
2.605841929204456
4.726648808616946
9.777316147082491
